# 1. Point projection and coordinate transformation

## Import Librarys

In [24]:
import cv2
import numpy as np
import os

## Import images

In [25]:


# Dynamische Pfaderstellung basierend auf dem aktuellen Notebook-Verzeichnis
notebook_dir = os.getcwd()
image_left_path = os.path.join(notebook_dir, 'Additional_files', 'Stereo_images', 'ConstructionSiteLeft', 'image0110_c0.pgm')
image_right_path = os.path.join(notebook_dir, 'Additional_files', 'Stereo_images', 'ConstructionSiteRight', 'image0110_c1.pgm')

image_left = cv2.imread(image_left_path, cv2.IMREAD_GRAYSCALE)
image_right = cv2.imread(image_right_path, cv2.IMREAD_GRAYSCALE)

# Kontrast für Anzeige erhöhen
image_left_display = cv2.normalize(image_left, None, 0, 255, cv2.NORM_MINMAX)
image_right_display = cv2.normalize(image_right, None, 0, 255, cv2.NORM_MINMAX)

print("image_left_path:", image_left_path)
print("image_right_path:", image_right_path)
print("image_left_display is None:", image_left_display is None)
print("image_right_display is None:", image_right_display is None)

image_left_path: c:\Users\lukas\Dokumente\Studium\Master\2_Semester\Faecher\Autonomous Systems Perception and Situation Understanding\ASPSU-Projects\Perception\Projekt\Additional_files\Stereo_images\ConstructionSiteLeft\image0110_c0.pgm
image_right_path: c:\Users\lukas\Dokumente\Studium\Master\2_Semester\Faecher\Autonomous Systems Perception and Situation Understanding\ASPSU-Projects\Perception\Projekt\Additional_files\Stereo_images\ConstructionSiteRight\image0110_c1.pgm
image_left_display is None: False
image_right_display is None: False


## Get Pixels by mouse click

In [26]:
USE_MOUSE = False
pixel_left = None
pixel_right = None

if USE_MOUSE:
    def mouse_callback(event, x, y, flags, param):
        global pixel_left, pixel_right
        if event == cv2.EVENT_LBUTTONDOWN:
            img, name = param
            print(f"{name}: Klick bei ({x}, {y}) ")
            if name == 'Left Image':
                pixel_left = [x, y]
            elif name == 'Right Image':
                pixel_right = [x, y]

    cv2.namedWindow('Left Image')
    cv2.setMouseCallback('Left Image', mouse_callback, param=(image_left, 'Left Image'))
    cv2.namedWindow('Right Image')
    cv2.setMouseCallback('Right Image', mouse_callback, param=(image_right, 'Right Image'))

    cv2.imshow('Left Image', image_left_display)
    cv2.imshow('Right Image', image_right_display)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

else:
    pixel_left = [250, 250]
    pixel_right = [238, 250]

print("pixel_left:", pixel_left)
print("pixel_right:", pixel_right)

pixel_left: [250, 250]
pixel_right: [238, 250]


## Calculate disparity

In [27]:
disparity = pixel_left[0] - pixel_right[0]
print("Disparity:", disparity)

Disparity: 12


## Read Camera parameters

In [29]:
camera_txt_path = os.path.join(notebook_dir, 'Additional_files', 'Camera.txt')
with open(camera_txt_path, 'r') as f:
    camera_txt_content = f.read()

internal_params = {}
external_params = {}

section = None
for line in camera_txt_content.strip().split('\n'):
    line = line.strip()
    if line == '[INTERNAL]':
        section = 'internal'
    elif line == '[EXTERNAL]':
        section = 'external'
    elif '=' in line and section:
        key, value = line.split('=', 1)
        value_clean = value.split('#', 1)[0].strip()
        try:
            value_num = float(value_clean)
        except ValueError:
            value_num = value_clean  # fallback if not a number
        if section == 'internal':
            internal_params[key.strip()] = value_num
        elif section == 'external':
            external_params[key.strip()] = value_num

print("internal_params:", internal_params)
print("external_params:", external_params)

internal_params: {'F': 820.428, 'SX': 1.0, 'SY': 1.000283, 'X0': 305.278, 'Y0': 239.826}
external_params: {'B': 0.308084, 'LATPOS': -0.07, 'HEIGHT': 1.26, 'DISTANCE': 2.0, 'TILT': 0.06, 'YAW': 0.01, 'ROLL': 0.0}


## Coordinate Calculation Camera


In [ ]:
# TODO  berechnung zu X, Y, Z
f = internal_params['F']
B = external_params['B']
z = (f * B) / disparity

pixel_hom = np.array([pixel_left[0], pixel_left[1], 1])
K_inv = np.linalg.inv(np.array([
    [internal_params['X0'], 0, internal_params['SX']],
    [0, internal_params['Y0'], internal_params['SY']],
    [0, 0, 1]
]))
xyz_norm = K_inv @ pixel_hom * z
x, y = xyz_norm[0], xyz_norm[1]
print("X:", x)
print("Y:", y)
print("Z:", z)

point_3d = np.array([x, y, z])

X: 17.180358080189208
Y: 21.869102570460324
Z: 21.063394996000003


## Shift from Camera to World

In [ ]:
# TODO: Umwandlung in DIN 70000 Koordinatensystem
point_3d_din = np.array([-point_3d[1], -point_3d[2], point_3d[0]])



print("3D-Punkt in Weltkoordinaten (DIN 70000):", point_3d_din)


3D-Punkt in Weltkoordinaten (DIN 70000): [-21.86910257 -21.063395    17.18035808]


## Rotation matrix calculation

In [ ]:
roll = external_params['ROLL']    # Rotation um X-Achse
pitch = external_params['TILT']   # Rotation um Y-Achse (hier als "TILT" bezeichnet)
yaw = external_params['YAW']      # Rotation um Z-Achse

# Rotationsmatrizen um die jeweiligen Achsen
Rx = np.array([
    [1, 0, 0],
    [0, np.cos(roll), -np.sin(roll)],
    [0, np.sin(roll), np.cos(roll)]
])

Ry = np.array([
    [np.cos(pitch), 0, np.sin(pitch)],
    [0, 1, 0],
    [-np.sin(pitch), 0, np.cos(pitch)]
])

Rz = np.array([
    [np.cos(yaw), -np.sin(yaw), 0],
    [np.sin(yaw), np.cos(yaw), 0],
    [0, 0, 1]
])

# Gesamte Rotationsmatrix: R = Rz * Ry * Rx (Z-Y-X-Konvention)
R = Rz @ Ry @ Rx

print("Rotationsmatrix R:\n", R)

Rotationsmatrix R:
 [[ 9.98150630e-01 -9.99983333e-03  5.99610083e-02]
 [ 9.98183903e-03  9.99950000e-01  5.99630071e-04]
 [-5.99640065e-02  0.00000000e+00  9.98200540e-01]]


## Shift vector

In [43]:
# Extrahiere den Verschiebungsvektor (Translation) aus den external_params
# LATPOS: Verschiebung in Y, HEIGHT: Verschiebung in Z, DISTANCE: Verschiebung in X (DIN 70000)
t = np.array([
    external_params['DISTANCE'],   # X
    external_params['LATPOS'],     # Y
    external_params['HEIGHT']      # Z
])

print("Verschiebungsvektor t:", t)

Verschiebungsvektor t: [ 2.   -0.07  1.26]


In [44]:
# Transformation des Punkts in das Weltkoordinatensystem mit Rotation und Verschiebung
point_3d_world = R @ point_3d_din + t
print("3D-Punkt in Weltkoordinaten (verschoben und rotiert):", point_3d_world)

3D-Punkt in Weltkoordinaten (verschoben und rotiert): [-18.58787648 -21.34033384  19.72080172]
